### Notebook to be used to perform manual focus testing using ACCS Images

In [ ]:
import numpy as np
#from lsst.ts import salobj
import asyncio
from astropy.io import fits

import warnings
#import matplotlib.pyplot as plt  # imported as py above
from astropy.modeling import models, fitting
from scipy.ndimage.filters import gaussian_filter as gauss_filt
from matplotlib import pyplot as plt

import copy
from pathlib import Path

In [ ]:
import numpy as np
import logging
logger = logging.getLogger('calc_CofM_logger')

def calc_CofM(array, offset=(0,0)):
    # Function takes a a 2d Array and computes the CofM
    
    # offset is the coordinate of the minimum y and x pixel
    
    
    #just make it easier to parse the code below
    shape=array.shape
        
    # Build 2d index arrays for calculations
    pix_index_1d_arr=np.arange(0, shape[0], 1)+ offset[0]
    ones_array_2d=np.ones([shape[1], shape[0]])
    y_index_arr_2d=np.transpose(pix_index_1d_arr*ones_array_2d)
    
    # Can't just use the transpose of the y_index_arr_2d
    # since we're supporting non-symmetrical arrays
    pix_index_1d_arr=np.arange(0, shape[1], 1) + offset[1]
    ones_array_2d=np.ones([shape[0], shape[1]])
    # array will be left to right so need to transpose
    x_index_arr_2d=pix_index_1d_arr*ones_array_2d

    # Calculate the centroid
    y_CofM = np.sum(y_index_arr_2d*array)/np.sum(array)
    x_CofM = np.sum(x_index_arr_2d*array)/np.sum(array)
    
    logger.info('y_CofM is {}'.format(y_CofM))
    logger.info('x_CofM is {}'.format(x_CofM))

    return(y_CofM, x_CofM)

In [ ]:
# Generate a set of test data
if False:
    array_size=(3520, 4656)
    psf_avg_pos = (array_size[0]/5.03, array_size[1]/3.05)
    psf_avg_sigma = float(18) # sigma in pixels
    max_pos_err= float(100.0) # maximum position random error in pixels
    max_sigma_err = float(3) # maximum variation in sigma
    amplitude = float(1000) #amplitude of gaussian
    dark_curr = 100
    n_frames = 11

    # Generate x,y,z data, where z is generated in the loop
    y, x = np.mgrid[:array_size[0], :array_size[1]]

    for n in np.arange(0,n_frames):
        # create base detector array
        im0 = np.zeros((array_size))+dark_curr
        # make Z array
        z=copy.deepcopy(im0)

        # put random offsets on parameters
        xoffset = max_pos_err*np.random.random()
        yoffset = max_pos_err*np.random.random()
        stddev_offset = max_sigma_err*np.random.random()

        # create data using astropy.modeling
        psf = models.Gaussian2D.evaluate(x,y, amplitude=amplitude, x_mean=psf_avg_pos[1]+xoffset,
                               y_mean=psf_avg_pos[0]+yoffset, 
                               x_stddev=psf_avg_sigma+stddev_offset,
                               y_stddev=psf_avg_sigma+stddev_offset,
                               theta=0.0)

        #add shot noise
        #psf+= (np.sqrt(psf)*np.random.randn(array_size[0], array_size[1]))

        # Display image?
        if False:
            fig2, ax2 = plt.subplots(figsize=(7,6))
            ax2.imshow(psf, interpolation='none')

        # Write the fits files
        hdu=None
        hdu = fits.PrimaryHDU(psf)
        hdul = fits.HDUList([hdu])
        fname = '20190910-generated-psf'+str(n)+'.fits'
        output_folder = Path("/home/saluser/data/20190910")
        print('writing psf file {}'.format(fname))
        hdul.writeto(output_folder / fname, overwrite=True)

In [ ]:
import os
os.environ["LSST_DDS_DOMAIN"]
import sys
import logging
import asyncio
from lsst.ts import salobj
import wget

In [ ]:
d = salobj.Domain()
gencam = salobj.Remote(d, 'GenericCamera', index=1)
await gencam.start_task
athexapod = salobj.Remote(d, 'ATHexapod')
await athexapod.start_task

In [ ]:
gencam.cmd_setLogLevel.set(level=logging.DEBUG)
athexapod.cmd_setLogLevel.set(level=logging.DEBUG)

In [ ]:
# Get summary state
#print(salobj.State(gencam.evt_summaryState.get().summaryState))
print(salobj.State(athexapod.evt_summaryState.get().summaryState))

In [ ]:
# Get current position by telemetry
# which is a list in X,Y,Z,U,V,Rot
await asyncio.sleep(1)
curr_hex_pos_telem = await athexapod.tel_positionStatus.next(flush = True, timeout=10)
print(curr_hex_pos_telem)

In [ ]:
# Move hexapod to desired position (in-focus)
# Note that minimal motions in X,Y,Z are ~0.3um (0.0003mm), so go to 4 decimal places
# minimal motions in U,V are ~3.5micro rads (0.0002 degrees), so go to 4 decimal places
# Rotation is not used
#hex_X, hex_Y, hex_Z, hex_U, hex_V, hex_R = [-4.29999921322, 1.19999866188-1000e-3, 0.50000155519-15e-3, 0.3500006508, 0.219999852315, -2.69867951921e-07]

hex_X, hex_Y, hex_Z, hex_U, hex_V, hex_R = [-4.29999921322-850e-3, 1.19999866188-1000e-3, 0.50000155519+60.0e-3, 0.3500006508, 0.219999852315, -2.69867951921e-07]


await athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
                                             z=hex_Z, u=hex_U, v=hex_V)

In [ ]:
# stop liveview
await gencam.cmd_stopLiveView.start()

In [ ]:
# r.evt_endReadout.flush()
await gencam.cmd_takeImages.set_start(numImages=50, expTime=0.3, shutter=True, imageSequenceName='psf')

tmp = await gencam.evt_endReadout.next(flush=False, timeout=1)
tmp = gencam.evt_endReadout.get()
print(tmp.imageName)

In [ ]:
wget_url = 'http://139.229.170.216:8000/data/'+tmp.imageName+'.fits'
print(wget_url)

In [ ]:
filename = wget.download(wget_url)
print('Grabbed/Wrote {} via wget'.format(filename))

In [ ]:
# Declare where data is located and where output will be written, notably the stacked images
data_folder = Path("/home/saluser/develop/ts_notebooks/pingraham/summit_notebooks/AT_20200108")
output_folder = Path("/home/saluser/data/output")

In [ ]:
im=(fits.open(data_folder / filename))[0].data

In [ ]:
print(np.max(im))

In [ ]:
if True:
    im_sub=im-np.median(im)
    plt.imshow(im_sub)

In [ ]:
# Find star by convolution with gaussian, then grab the max
tmp=gauss_filt(im,[5,5],mode='constant',cval=0)
print(np.max(tmp))
value=None
ind=np.argmax(tmp)
ind2d = np.unravel_index(ind,tmp.shape)
print('centroid at y,x: {} {}'.format(ind2d[0],ind2d[1]))
ind2d0=copy.deepcopy(ind2d)

if False:
    ind2d=np.array((362, 416))
    print('OVERRIDING CENTROID to use')

#ymin=1800 ; ymax=2048
#xmin=280 ; xmax=450
half_side=int(20)
ymin = ind2d[0]-half_side if ind2d[0]-half_side > 0 else 0
ymax = ind2d[0]+half_side if ind2d[0]+half_side < im.shape[0] else im.shape[1]
xmin = ind2d[1]-half_side if ind2d[1]-half_side > 0 else 0
xmax = ind2d[1]+half_side if ind2d[1]+half_side < im.shape[1] else im.shape[1]
print('subimage being made from ymin,ymax,xmin,xmax:{} {} {} {}'.format(ymin,ymax,xmin,xmax))
# new maximum at:
ind=np.argmax(tmp[ymin:ymax,xmin:xmax])
ind2d = np.unravel_index(ind,tmp[ymin:ymax,xmin:xmax].shape)
print('new centroid at y,x: {} {}'.format(ind2d[0],ind2d[1]))

In [ ]:
im.shape
yoffset_pix=im.shape[0]/2 - ind2d0[0]
xoffset_pix=im.shape[1]/2 - ind2d0[1]
print('yoffset = {} pixels'.format(yoffset_pix))
print('xoffset = {} pixels'.format(xoffset_pix))

In [ ]:
binning=4
plate_scale = 0.1 *1000 # um/arcsec
pix_scl = 3.6*binning # um/pix

yoffset_as = yoffset_pix * pix_scl / plate_scale
xoffset_as = xoffset_pix * pix_scl / plate_scale
print('yoffset is {} arcsec'.format(yoffset_as))
print('xoffset is {} arcsec'.format(xoffset_as))

In [ ]:
im_sub=im[ymin:ymax,xmin:xmax]
im_sub=im_sub-np.median(im_sub)
plt.imshow(im_sub)
print(np.nanmax(im_sub))

In [ ]:
im_sub=im[ymin:ymax,xmin:xmax]
im_sub=im_sub-np.median(im_sub)
plt.imshow(im_sub)
print(np.nanmax(im_sub))

In [ ]:
# Generate x,y,z data
y, x = np.mgrid[:im_sub.shape[0], :im_sub.shape[1]]
print(y[ind2d])
print(x[ind2d])
z=im_sub
# Fit the data using astropy.modeling
p_init = models.Gaussian2D(amplitude=np.nanmax(z),x_mean=x[ind2d], y_mean=y[ind2d], fixed={'theta':False})
#p_init = models.Gaussian2D(amplitude=np.nanmax(z),x_mean=38, y_mean=51, x_stddev=3, y_stddev=3,  fixed={'theta':True})
fit_p = fitting.LevMarLSQFitter()

with warnings.catch_warnings():
    # Ignore model linearity warning from the fitter
    warnings.simplefilter('ignore')
    p = fit_p(p_init, x, y, z)

# Plot the data with the best-fit model
plt.figure(figsize=(9, 5))
plt.subplot(1, 3, 1)
plt.imshow(z, origin='lower', interpolation='nearest')#, vmin=-1e4, vmax=5e4)
plt.title("Data")
plt.subplot(1, 3, 2)
plt.imshow(p(x, y), origin='lower', interpolation='nearest')#, vmin=-1e4, vmax=5e4)
plt.title("Model")
plt.subplot(1, 3, 3)
plt.imshow(z - p(x, y), origin='lower', interpolation='nearest')#, vmin=-1e4, vmax=5e4)
plt.title("Residual")

In [ ]:
repr(p)

In [ ]:
# FWHM [arcsec] = 3.35 [pix] * 10 [arcsec/mm] * 3.6e-3 [mm/pix]* 4 * 2.355 = 1.1 arcseconds (DIMM says ~0.9 arcsec)
max_stddev_axis = np.max((p.x_stddev.value, p.y_stddev.value))
print('Seeing = {} [arcsec]'.format(max_stddev_axis * 10 * 3.6e-3 * binning * 2.355))

In [ ]:
binning

In [ ]:
print('{}, {}, {}'.format(psf_fname, p.x_stddev.value, p.y_stddev.value))

In [ ]:
# hexapod Z, filename, xsttd, ystddev
# 0.300, 1325429366-bet_grus-0-1.fits, 20.17756577291818, 18.49931466099071
# 0.032, 1325429366-bet_grus-0-1.fits, 18.605872647081075, 14.995560001304131
# 0.33, 1325429366-bet_grus-0-1.fits, 17.535626208313843, 15.471926526885643
# 0.35, 1325429366-bet_grus-0-1.fits, 16.925439176728673, 14.719989238436273
# 0.34, 


In [ ]:
# Make an radial plot
pix_index_1d_arr=np.arange(0,2*half_side,1)
ones_array_2d=np.ones([2*half_side,2*half_side])
#print(x_arr)
#print(tmp)
x_index_arr_2d=pix_index_1d_arr*ones_array_2d
x_arr_2d=x_index_arr_2d - p.x_mean.value
y_arr_2d= np.transpose(x_index_arr_2d) - p.y_mean.value
# create array of radial distances (in pixels)
r_arr_2d=np.sqrt(x_arr_2d**2 + y_arr_2d**2)

In [ ]:
if False:
    plt.imshow(r_arr_2d)
    plt.colorbar()

In [ ]:
r_arr_1d=np.reshape(r_arr_2d,(2*half_side)**2)
values = np.reshape(z,(2*half_side)**2)

In [ ]:
if True:
    plt.ylabel('Intensity (ADU)')
    plt.plot(r_arr_1d, values,'.')
    plt.title('Radial PSF')
    plt.xlabel('radius [pix]')
    plt.show()
    plt.close()